# dlt assessment - Jaffle API ingestion

1. Describe how you would use dlt in a previous job or project. If you don’t have a real example, pick a realistic one (e.g., analytics platform, marketing data, finance reporting, product telemetry). Aim for 5–10 sentences, focused on clarity and trade-offs.

2. Build a dlt pipeline, using our Jaffle Shop API as a source and DuckDB as a destination.
Use the API spec for this API and the “https://jaffle-shop.dlthub.com/api/v1” as an API base URL
Make sure you use some more advanced dlt functionality like the merge write disposition and incremental loading.

3. Assume this pipeline is the starting point for a real engagement with a client. Describe how you would take it to production.
List the key questions you’d ask before recommending a setup.
Highlight the aspects the client needs to pay attention to. You don’t need to go super deep, highlight one or two aspects you would ask the client to consider when running this pipeline in production.
What would you recommend for the best practices for dlt code & deployment. You can talk about env & configuraiton, testing and validation, CI/CD, orchestration & runtime.

# 1. dlt use cases

1. Unified Ads Performance Dashboard (Facebook & Google)

**Problem**

A marketing company using direct connectors ( Supermetrics) for Facebook and Google Ads faced expensive licensing fees and slow dashboards.
They lacked a centralized database.

**Objective**

Replace the slow connectors with a dlt-based ELT pipeline that ingests raw data directly into BigQuery. This aimed to unify ad spend metrics into a single, high-performance SQL view for fast and accurate reporting in Looker Studio.

**Limits**

No data people in the team to follow up or create the SQL requests necessary for the dashboard. Company not data mature to consider the need of a data pipeline.

2. Lightweight API Ingestion for a Solo Analyst

**Problem**

A solo data analyst needed to centralize data from five distinct APIs into Snowflake but faced budget constraints that ruled out managed SaaS solutions like Airbyte Cloud. His attempt to self-host Airbyte on Kubernetes (GKE) proved operationally overwhelming and too complex for a one-person team to maintain.

**Objective**

The goal was to replace heavy infrastructure with a lightweight dlt-based pipeline that runs on simple CLoud Function. This approach aimed to bypass the complexity of managing a self-hosted platform.

**Limits**

The data Analyst were not very comfortable to manage a python ingestion pipeline by himself.


# 2. Build a dlt pipeline for Jaffle API


### Gettings source informations

In [52]:
# first we need to get all endpoints infos
!curl https://jaffle-shop.dlthub.com/docs


    <!DOCTYPE html>
    <html>
    <head>
    <link type="text/css" rel="stylesheet" href="https://cdn.jsdelivr.net/npm/swagger-ui-dist@5/swagger-ui.css">
    <link rel="shortcut icon" href="https://fastapi.tiangolo.com/img/favicon.png">
    <title>dltHub Jaffle Shop API - Swagger UI</title>
    </head>
    <body>
    <div id="swagger-ui">
    </div>
    <script src="https://cdn.jsdelivr.net/npm/swagger-ui-dist@5/swagger-ui-bundle.js"></script>
    <!-- `SwaggerUIBundle` is now available on the page -->
    <script>
    const ui = SwaggerUIBundle({
        url: '/openapi.json',
    "dom_id": "#swagger-ui",
"layout": "BaseLayout",
"deepLinking": true,
"showExtensions": true,
"showCommonExtensions": true,
oauth2RedirectUrl: window.location.origin + '/docs/oauth2-redirect',
    presets: [
        SwaggerUIBundle.presets.apis,
        SwaggerUIBundle.SwaggerUIStandalonePreset
        ],
    })
    </script>
    </body>
    </html>
    

In [53]:
import dlt
from dlt.sources.helpers import requests
import duckdb
import json

# the url is '/openapi.json', so lets create a local json to have all infos available.
api_doc_url = "https://jaffle-shop.dlthub.com/openapi.json"

api_doc_json = requests.get(api_doc_url).json()
print(api_doc_json)

{'openapi': '3.1.0', 'info': {'title': 'dltHub Jaffle Shop API', 'description': 'The dltHub Jaffle Shop API is a RESTful API that provides access to the dbt Jaffle Shop dataset. The code for this API is open source and available at https://github.com/dlt-hub/fast-api-jaffle-shop.', 'contact': {'name': 'dltHub', 'url': 'https://dlthub.com/', 'email': 'support@dlt.hub'}, 'license': {'name': 'APACHE', 'url': 'https://www.apache.org/licenses/LICENSE-2.0.html'}, 'version': '1.0.0'}, 'paths': {'/api/v1/customers': {'get': {'tags': ['customers'], 'summary': 'Get Customers', 'description': 'Returns a paginated list of customers.\n\nPagination is controlled via the `page` query parameter (see `page`).\n\nEach page returns a fixed number of results (see `page_size`).\n\nIf more results are available, the response will include a `Link` header with `rel="next"` that points to the next page.', 'operationId': 'get_customers_api_v1_customers_get', 'parameters': [{'name': 'page', 'in': 'query', 'requi

In [54]:
# by storing the entier json doc, we will have a more precise overview of endpoints and parameters. 
# We could potentially feed an LLM to write a markdowon human friendly documentation.

with open('jaffle_api_doc.json', 'w') as doc:
    json.dump(api_doc_json, doc)

### Create the source ressources

In [55]:
# using the .config file to store environment variables
base_url = dlt.config["runtime.BASE_URL"]
print(base_url)

https://jaffle-shop.dlthub.com/api/v1


### Creation Jaffle APi source by creating the ressources

In [56]:
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import HeaderLinkPaginator

# list of existing endpoints for potential dynamic ressources creation
endpoints = ["customers","orders","items","products","supplies","stores", "row_counts" ]
page_size = 100 # page size is the same across the API

@dlt.source
def get_jaffle_api(base_url: str = base_url, page_size: int = page_size):
    # we initialise the client once because they share the same params
    client = RESTClient(
        base_url=base_url,
        paginator=HeaderLinkPaginator(links_next_key="next"), # based on api doc, we use the header link
    )

    @dlt.resource(name='customers', write_disposition='append') # endpoint gives an id and a name. Stateful data here.
    def get_customers():
        """Retrieves customers from the Jaffle Shop API"""
        for page in client.paginate('customers',params={"page_size": page_size}):
            yield page
    
    @dlt.resource(name='orders', write_disposition='append') # doesnt have any 'updated_at' column so append is the right way to do. 
    def get_orders(page_size: int = page_size):
        """Retrieves orders from the Jaffle Shop API."""

        for page in client.paginate('orders',params={"page_size": page_size}):
            yield page


    @dlt.resource(name='items', write_disposition='append') #items associated to an order. many to many relation. we need to append here
    def get_items(page_size: int = page_size):
        """Retrieves items from the Jaffle Shop API."""

        for page in client.paginate('items',params={"page_size": page_size}):
            yield page

    @dlt.resource(name='products', write_disposition='merge', primary_key='sku') #products database, probably not updated often but important to catch those changes. we changed the PK to sku
    def get_products(page_size: int = page_size):
        """Retrieves products from the Jaffle Shop API."""

        for page in client.paginate('products',params={"page_size": page_size}):
            yield page


    @dlt.resource(name='supplies', write_disposition='merge')
    def get_supplies(page_size: int = page_size):
        """Retrieves supplies from the Jaffle Shop API."""

        for page in client.paginate('supplies',params={"page_size": page_size}):
            yield page


    @dlt.resource(name='stores', write_disposition='merge' )
    def get_stores(page_size: int = page_size):
        """Retrieves stores from the Jaffle Shop API."""
        
        for page in client.paginate('stores',params={"page_size": page_size}):
            yield page

    @dlt.resource # optional. useful for comparing data source rows and ingested rows
    def get_row_counts(name='row_counts'):
        """Rerieves row counts from the Jaffle Shop API."""
        
        for page in client.paginate('row-counts',params={"page_size": page_size}):
            yield page


    return (get_customers, get_orders, get_items, get_products, get_supplies, get_stores, get_row_counts)

### Pipeline run 

In [57]:
pipeline = dlt.pipeline(
    pipeline_name="jaffle_pipeline",
    destination="duckdb",
    dataset_name="raw_data",
    dev_mode=True, # allow us to debug while building this PoC. to remove before going to production
    refresh=True
)

load_info = pipeline.run(get_jaffle_api(), write_disposition='merge')
print(load_info)

2026-02-02 20:28:41,435|[WARNING]|2948|8822055488|dlt|utils.py|verify_schema_merge_disposition:161|Table items has `write_disposition` set to `merge` and `merge_strategy` set to `delete-insert`, but no primary or merge keys defined. dlt will fall back to `append` for this table.
2026-02-02 20:28:41,436|[WARNING]|2948|8822055488|dlt|utils.py|verify_schema_merge_disposition:161|Table customers has `write_disposition` set to `merge` and `merge_strategy` set to `delete-insert`, but no primary or merge keys defined. dlt will fall back to `append` for this table.
2026-02-02 20:28:41,436|[WARNING]|2948|8822055488|dlt|utils.py|verify_schema_merge_disposition:161|Table stores has `write_disposition` set to `merge` and `merge_strategy` set to `delete-insert`, but no primary or merge keys defined. dlt will fall back to `append` for this table.
2026-02-02 20:28:41,437|[WARNING]|2948|8822055488|dlt|utils.py|verify_schema_merge_disposition:161|Table orders has `write_disposition` set to `merge` and 

Pipeline jaffle_pipeline load step completed in 10.06 seconds
1 load package(s) were loaded to destination duckdb and into dataset raw_data_20260202072752
The duckdb destination used duckdb:////Users/macbook/Development/jaffle_poc/jaffle_pipeline.duckdb location to store data
Load package 1770060472.608057 is LOADED and contains no failed jobs


### Observation of the results

In [58]:
# lets check our results

customers = pipeline.dataset().customers.df()
orders = pipeline.dataset().orders.df()
items= pipeline.dataset().items.df()
products = pipeline.dataset().products.df()
supplies = pipeline.dataset().supplies.df()
stores = pipeline.dataset().stores.df()

orders

,id,customer_id,store_id,ordered_at,subtotal,tax_paid,order_total,_dlt_load_id,_dlt_id
0,9bed808a-5074-4dfb-b1eb-388e2e60a6da,50a2d1c4-d788-4498-a6f7-dd75d4db588f,4b6c2304-2b9e-41e4-942a-cf11a1819378,2016-09-01 15:01:00+00:00,700.0,42.0,742.0,1770060472.608057,q6aeJ6eCbnVfmA
1,b83630c1-0fdc-4cd2-818d-0b6d4384ce86,438005c2-dd1d-48aa-8bfd-7fb06851b5f8,4b6c2304-2b9e-41e4-942a-cf11a1819378,2016-09-01 10:39:00+00:00,700.0,42.0,742.0,1770060472.608057,y6bZpGwhFGHPvA
2,3b4a03db-7b23-4673-a88a-7f51b01ca497,5261268c-aa94-438a-921a-05efc0d414ac,4b6c2304-2b9e-41e4-942a-cf11a1819378,2016-09-01 07:46:00+00:00,700.0,42.0,742.0,1770060472.608057,taW/PdrBfz1Bow
3,3368b213-6687-4338-bf73-b927ae72340f,f8486fce-bc07-4a4f-a6e9-ed6a06ba996c,4b6c2304-2b9e-41e4-942a-cf11a1819378,2016-09-01 14:39:00+00:00,600.0,36.0,636.0,1770060472.608057,pRaXD27NgHITEQ
4,739bf2f3-8d20-4159-9124-5451f1e4b136,341ed9b2-1760-4720-a1b1-42681d273c63,4b6c2304-2b9e-41e4-942a-cf11a1819378,2016-09-01 10:57:00+00:00,1100.0,65.0,1165.0,1770060472.608057,PUtf+8oB73oZ9g
...,...,...,...,...,...,...,...,...,...
61943,d655d186-275f-4411-9a23-2f6d68f97578,2d4e6ed6-f081-4049-b92a-7792dbe97b89,40e6ddd6-b8f6-4e17-8bd6-5e53966809d2,2017-08-31 11:31:00+00:00,600.0,24.0,624.0,1770060472.608057,ip4d2Lpet/Gsuw
61944,f6d1a004-6d02-4846-974c-aa48891cf1e1,3d6f18db-a49c-4563-9445-68d4e02bd664,40e6ddd6-b8f6-4e17-8bd6-5e53966809d2,2017-08-31 08:14:00+00:00,600.0,24.0,624.0,1770060472.608057,qlBRQejfAo6YLA
61945,505da4ed-218d-47ce-bb62-caadac5da974,e8014144-a99b-4ca2-8418-56404aee2f08,40e6ddd6-b8f6-4e17-8bd6-5e53966809d2,2017-08-31 16:07:00+00:00,600.0,24.0,624.0,1770060472.608057,btqBV+H/wrfENA
61946,1b41551c-b925-4cf5-820f-b014a23015a0,8f054a96-7920-42dc-b286-f6980ea8dbeb,40e6ddd6-b8f6-4e17-8bd6-5e53966809d2,2017-08-31 17:41:00+00:00,1200.0,48.0,1248.0,1770060472.608057,M/IAiIlOSASbkw


In [59]:
row_counts = pipeline.dataset().get_row_counts.df()
row_counts

,table_name,row_count,_dlt_load_id,_dlt_id
0,customers,935,1770060472.608057,8QOzLUjH6cBTxQ
1,orders,61948,1770060472.608057,Tb2Hay1uLLarqw
2,items,90900,1770060472.608057,xIkbovpVXgB2CQ
3,products,10,1770060472.608057,JlgjRa25OKg7zQ
4,stores,6,1770060472.608057,cy08CiyGJZ02Hg
5,supplies,65,1770060472.608057,j27WyAL+cxyHZw


---

# 3. Taking the Pipeline to Production

## Phase 1: Infrastructure & Team Audit

I perform this assessment before even writing a single line of production code.
1. Business Value & Strategy
    Goal: Understand the ROI to justify the maintenance cost and ensure the pipeline is an asset, not a liability.
    The "Why": Verify the actual business use case beyond simply "having the data."

    Data Freshness & Lifecycle:

    - SLAs: Is the data needed in real-time (streaming), hourly, or daily?
    - Retention: Define a Hot/Cold storage strategy. How long must data be queryable before archiving?
    - Disaster Recovery: Is there a backup/restore plan for the warehouse?

2. The team

Who am I serving?
- Target Audience: Identify the primary consumers (Data Analysts, Scientists, Stake holders).
- Technical Proficiency: Are they SQL-native? How confortable in Python?
- Tooling Familiarity: Do they prefer Snowflake, BigQuery, Looker, Streamlit...?
- Partnership: Identify a Data referent in the business team who can validate data quality on the long run.

3. Governance & Compliance
- PII Management: How do we obfuscate or hash sensitive data (GDPR compliance)?`
- Data Residency: specific requirements for where data is stored (e.g., EU-only servers).
- IAM & Security: Define role-based access control. Who can modify the pipeline vs. who can read the dataset?

## Phase 2: Technical Execution
1. Architecture & Design
    - Diagramming: Create a high-level architecture diagram (e.g., using Excalidraw) to visualize flow and dependencies.
    - Infrastructure Selection:
    - Greenfield: For new setups, I recommend MotherDuck efficient & fast, easy deployment from Duckdb and EU server options.
    - Other: Adapt to the client's existing stack (Snowflake/BigQuery).

2. Deployment & Orchestration

    - Compute: decide where the python code runs.
        - Simple: GitHub Actions with  cron jobs (Keep It Super Simple) 
        OR
        - Scalable: Astro or Dagster (not a big fan of Cloud composer)
    - Secrets Management: Never store keys in code. Use GitHub Secrets.
    - CI/CD: Implement automated testing when deploying pipelines.

3. Source Reliability
    - Implement unit test for data type checking and null-value. It could be also done in SQL via dbt unit tests
    - API settings: Does the source support incremental loading (cursors/timestamps) or webhooks?
    - Rate limiting: Implement backoff strategies to respect API quotas.

4. Documentation & Observability
    - Lineage: Document the data transformation flow.
    - Alerting: Set up notifications (Slack/email) for failures.
    - Dcoumentation access: Ensure documentation is updated automatically or easy to find the Source of truth.